#Notebooks Basics

In [0]:
print("hello world")

In [0]:
%sql
select "RUN SQL"

## Database Objects 
Create following objects 
1. Catalog 
2. Schema
3. Table
4. Volume
5. Function

In [0]:
%sql
create table dev.yash.mytable (id int)

In [0]:
%sql describe table dev.yash.mytable

In [0]:
%sql
insert into dev.yash.mytable values (1),(2),(3),(4),(5)

In [0]:
%sql
select * from dev.yash.mytable

In [0]:
%sql
create volume dev.yash.myvolume;
    
describe volume dev.yash.myvolume;

In [0]:

cp /Volumes/dev/naval/raw/1.json /Volumes/dev/yash/myvolume/1.json

In [0]:
%sql
describe schema dev.yash;

In [0]:
%sql
create or replace function dev.yash.full_name(first_name string, last_name string) returns string
return concat(first_name, ' ', last_name)

In [0]:
%sql
select dev.yash.full_name('Yaswanth Raj Kumar', "Yadiki")

##Ways of create a table
1. pyspark dataframe style
2. CTAS - Spark sql
  i. select * from file_format.path(single json, parquet, detla)
  ii. select * from read_files("path",format=>"csv") (single json, parquet, detla, csv, mutliline json, etc etc)
3. UI (limiatation)

In [0]:
%sql
-- Through Json
use catalog dev;
create or replace table yash.medical_data as 
select * from json.`/Volumes/dev/yash/myvolume/1.json`

In [0]:
%sql
select * from parquet.`/Volumes/dev/naval/raw/part-00000-7951754c-3166-4cb7-8227-a5760c654c14-c000.snappy.parquet`

In [0]:
%sql
select * from csv.`/Volumes/dev/naval/raw/financial_dataset.csv`
-- This approach pulls header also as a row and doesn't define the data type

In [0]:
%sql
create or replace table dev.yash.financial_data as
select *, current_date() as ingestion_date from read_files("/Volumes/dev/naval/raw/financial_dataset.csv",format=>"csv")

In [0]:
%sql
select *, current_date() as ingestion_date, _metadata.file_path as metadata from read_files("/Volumes/dev/naval/raw/financial_dataset.csv",format=>"csv")

## Lazy Evalution

In [0]:
df = spark.read.json("/Volumes/dev/yash/myvolume/1.json")

In [0]:
df.write.mode("overwrite").saveAsTable("dev.yash.medical_py_df")

In [0]:
%sql
desc extended dev.yash.medical_py_df;

### Task
- Step 1: create a Dataframe by read the csv file
- Step 2: Tranformation
        - i. rename TransactionID to transaction_id
        - ii. rename CustomerID to customer_id
        - iii. add new col ingestion_date using current_timestamp 
        - iv. add new col metadata using _metadatat.filename
- Step 3: save it to a table

In [0]:
%sql
drop table dev.yash.financial;

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import current_timestamp
schema = StructType([
    StructField("TransactionID", StringType(), True),
    StructField("CustomerID", StringType(), True),
    StructField("AccountType", StringType(), True),
    StructField("TransactionType", StringType(), True),
    StructField("AmountUSD", DoubleType(), True),
    StructField("Branch", StringType(), True),
    StructField("TransactionDate", DateType(), True)
])
df = spark.read.schema(schema).csv("/Volumes/dev/naval/raw/financial_dataset.csv", header=True)
df = (df.withColumnRenamed("TransactionID", "transaction_id")
      .withColumnRenamed("CustomerID", "customer_id")
      .withColumn("ingestion_date", current_timestamp())
      )
df.write.mode("overwrite").saveAsTable("dev.yash.financial")

In [0]:
%sql
select * from dev.yash.financial;